In [19]:
import pandas as pd
from sodapy import Socrata
import folium
from folium.features import GeoJson
import math

from cross_arc_distance import cross_arc_distance

import secrets

In [21]:
## Initialize the client
client = Socrata("www.data.act.gov.au",
            app_token=secrets.app_token,
            username=secrets.username,
            password=secrets.password)

## You can initialize the client in "severely rate-limited" form with the following
# client = Socrata("www.data.act.gov.au", None)

street_light_dataset = client.get("n9u5-bt96", limit=100000)
all_paths_dataset = client.get("ee7d-h7nm", limit=100000)

# Paths & Lights Data

In [80]:
all_paths_df = pd.DataFrame.from_records(all_paths_dataset)
bike_paths_df = all_paths_df[all_paths_df['path_type']=='CYCLEPATH']
pedestrian_paths_df = all_paths_df[all_paths_df['path_type']=='FOOTPATH']

street_light_df = pd.DataFrame.from_records(street_light_dataset)
street_light_df['close_to_path'] = False

In [81]:
cutoff = 5

for _idx, bike_path_entry in bike_paths_df.iterrows():
    point_pair_list = bike_path_entry['the_geom']['coordinates'][0]
    for idx, _point_pair in enumerate(point_pair_list):
        for light_idx, street_light_entry in street_light_df.iterrows():

            if street_light_df.loc[light_idx, 'close_to_path']:
                continue

            if idx > 0:
                current_point = point_pair_list[idx]
                previous_point = point_pair_list[idx-1]

                ## Point A, radians
                previous_point_lat = float(previous_point[1]) * math.pi/180
                previous_point_long = float(current_point[0]) * math.pi/180
                
                ## Point B, radians
                current_point_lat = float(current_point[1]) * math.pi/180
                current_point_long = float(current_point[0]) * math.pi/180

                ## Point C, radians
                light_latitude = float(street_light_entry['location']['latitude']) * math.pi/180
                light_longitude = float(street_light_entry['location']['longitude']) * math.pi/180

                nearest_distance_path_to_light = cross_arc_distance(
                        previous_point_lat, previous_point_long,
                        current_point_lat, current_point_long,
                        light_latitude, light_longitude
                        )

                if nearest_distance_path_to_light < cutoff:
                    ## Set light dataframe near_to_path to True and break
                    street_light_df.loc[light_idx, 'close_to_path'] = True


In [ ]:
street_light_close_to_path_df = street_light_df[street_light_df['close_to_path']==True]

In [ ]:
## coordinates from Canberra wiki
initial_lat = -35.473469
initial_lng = 149.012375

# Load map centred on average coordinates
my_map = folium.Map(location=[initial_lat, initial_lng], zoom_start=10, tiles="cartodbpositron")

## Style definitions
bike_path_style = {'color': '#228B22', 'fillOpacity': 0.5}

for _idx, bike_path_entry in bike_paths_df.iterrows():
    GeoJson(
        data = bike_path_entry['the_geom'],
        name = 'bike_path',
        style_function = lambda x : bike_path_style
    ).add_to(my_map)

for _idx, street_light_entry in street_light_close_to_path_df.iterrows():
    folium.CircleMarker(
        location = [float(street_light_entry['location']['latitude']), float(street_light_entry['location']['longitude'])],
        radius = 1, ##0.2
        color = "red", 
        fill = True,
        fill_color = "red",
        fill_opacity = 0.7
    ).add_to(my_map)

my_map